<a href="https://colab.research.google.com/github/thomasguizzetti/projects/blob/master/MIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Installing and importing Z3

In [ ]:
!pip install pyomo
!apt-get install -y -qq glpk-utils
!apt-get install -y -qq coinor-cbc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.5 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 122545 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-2_amd64.deb ...
Unpacking libgl

In [ ]:
!python -m pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 69.7 MB/s eta 0:00:00


###importing the variables from the text file

In [ ]:
def import_variables(name):
  with open(name, 'r') as f:
    # Read the first line and convert it to an integer
    first_line = int(f.readline().strip())
    
    # Read the second line and convert it to an integer
    second_line = int(f.readline().strip())
    
    # Read the next two lines and convert them to arrays
    array1 = list(map(int, f.readline().strip().split()))
    array2 = list(map(int, f.readline().strip().split()))
    
    # Read the rest of the lines and convert them to a two-dimensional array
    two_d_array = []
    for line in f:
        row = list(map(int, line.strip().split()))
        two_d_array.append(row)

  # number of couriers
  m = first_line

  # number of items
  n = second_line

  # maximum load size of each courier
  l = array1

  # each item's size
  s = array2

  # Distance between distribution point i
  # and distribution point j (each items destination)
  D = two_d_array

  return m, n, l, s, D

## Input Examples

Just Run one to use the model

Text File

In [ ]:
# m is the number of couriers
# n is the number of items
# l is the capacity of couriers
# s is the sizeof each item
# d is the distances between destinations
m, n, l, s, d = import_variables("inst08.dat")


m, n, l, s, d

(8,
 10,
 [180, 160, 185, 200, 180, 160, 175, 200],
 [10, 25, 18, 16, 14, 7, 9, 16, 24, 19],
 [[0, 56, 86, 87, 81, 128, 107, 163, 166, 98, 93],
  [56, 0, 80, 31, 63, 87, 61, 107, 119, 76, 37],
  [86, 71, 0, 102, 17, 42, 111, 77, 90, 12, 70],
  [87, 31, 110, 0, 93, 116, 30, 77, 89, 105, 40],
  [81, 63, 17, 93, 0, 47, 94, 82, 94, 17, 53],
  [128, 87, 42, 116, 47, 0, 117, 52, 92, 30, 76],
  [117, 61, 111, 30, 94, 117, 0, 65, 59, 106, 41],
  [163, 107, 77, 77, 82, 52, 65, 0, 40, 65, 70],
  [166, 110, 90, 79, 94, 92, 49, 40, 0, 82, 82],
  [98, 76, 12, 105, 17, 30, 106, 65, 82, 0, 65],
  [93, 37, 70, 40, 53, 76, 41, 70, 82, 65, 0]])

In [ ]:
from pyomo.environ import *
import numpy as np

# Create a ConcreteModel
model = ConcreteModel()

# Define the decision variable matrix
# i is the courier and j is the item
model.x = Var(range(m), range(n), within=Boolean)
model.roots = Var(range(m), range(n+1), range(n+1), within=Boolean)

# Define the MTZ variables
model.u = Var(range(m), range(n+1), within=NonNegativeIntegers)

# Define the objective function to minimize the total distance traveled by the couriers
model.max_distance = Var(within=NonNegativeIntegers)

model.distance = Var(within=NonNegativeIntegers)


# Define the solver
solver = SolverFactory('gurobi')
#solver = SolverFactory('glpk')

# Define the constraints
model.constraints = ConstraintList()

# 1) The sum of each row cannot be more than 1
# because we cannot go to multiple places at once
for i in range(m):
    for j in range(n+1):
        model.constraints.add(sum(model.roots[i, j, k] for k in range(n+1)) <= 1)

# 2) The sum of no column can be more than 1
# because we don't want subtours
for i in range(m):
    for k in range(n+1):
        model.constraints.add(sum(model.roots[i, j, k] for j in range(n+1)) <= 1)

# 3) Enforce the constraint that the sum of the first column of each matrix is exactly 1
# because we're obligated to come back to the point 0
for i in range(m):
    model.constraints.add(sum(model.roots[i, j, n] for j in range(n+1)) == 1)
    model.constraints.add(sum(model.roots[i, n, j] for j in range(n+1)) == 1)

#Add the MTZ constraint
for i in range(m):
    for j in range(n+1):
        for k in range(n+1):
            if j != n and j != k:
                model.constraints.add(model.u[i, j] - model.u[i, k] + n * model.roots[i, j, k] <= n - 1)

# 5) Set diagonal elements of roots to 0
for i in range(m):
    for j in range(n+1):
        model.constraints.add(model.roots[i, j, j] == 0)


# Define the capacity constraint, which ensures that each item is assigned to exactly one courier
for j in range(n):
    # The summation of the number of couriers responsible for taking one package shouldn't surpass 1
    model.constraints.add(sum(model.x[i, j] for i in range(m)) == 1)

# Define the capacity constraint, which ensures that each courier does not exceed their capacity
for i in range(m):
    model.constraints.add(
        sum(model.x[i, j] * s[j] for j in range(n)) <= l[i]
    )

# Connecting roots to x: every x which is assigned should be visited by roots
for i in range(m):
    for j in range(n):
        model.constraints.add(sum(model.roots[i, j, k] for k in range(n+1)) == model.x[i, j])
        model.constraints.add(sum(model.roots[i, k, j] for k in range(n+1)) == model.x[i, j])


# Define the objective function to minimize the total distance traveled by the couriers
for i in range(m):
    model.constraints.add(sum(model.roots[i, j, k] * d[j][k] for j in range(n+1) for k in range(n+1)) <= model.max_distance)

model.constraints.add(sum(model.roots[i, j, k] * d[j][k] for i in range(m) for j in range(n+1) for k in range(n+1)) == model.distance)

# Define the objective function to minimize the total distance traveled by the couriers
# model.objective = Objective(expr=model.max_distance, sense=minimize)
# model.objective = Objective(expr=model.distance, sense=minimize)
model.objective = Objective(expr=model.max_distance + model.distance, sense=minimize)


# Set solver options
# solver.options['timelimit'] = 60  # Set a time limit of 3600 seconds
# solver.options['mipgap'] = 0.001  # Set the MIP optimality gap tolerance to 1%
# # Set solver options
# solver.options['MIPFocus'] = 2  # Focus on finding good feasible solutions
# solver.options['Heuristics'] = 0.8  # Allow more time for heuristics to find better solutions
# solver.options['Threads'] = 2  # Use multiple threads for parallel processing


# Solve the optimization problem
results = solver.solve(model)


# Check the solver status and termination condition
if (results.solver.termination_condition == TerminationCondition.optimal or
        results.solver.termination_condition == TerminationCondition.locallyOptimal):
    items = [[] for _ in range(m)]
    X = []

    for i in range(m):
        for j in range(n):
            X.append(value(model.x[i, j]))
            if value(model.x[i, j]):
                items[i].append(j)

    print("Items:", items)
    print("Assignment Matrix:")
    print(np.array(X).astype(int).reshape([m, n]))

    courier_matrices = []
    for i in range(m):
        courier_matrix = []
        print(f"Courier {i}:")
        for j in range(n+1):
            row = []
            for k in range(n+1):
                row.append(value(model.roots[i, j, k]))
            print(np.array(row).astype(int))
            courier_matrix.append(np.array(row).astype(int))
        courier_matrix = np.concatenate(courier_matrix, axis=0).reshape([n+1, n+1])
        courier_matrices.append(courier_matrix)

    print("MTZ Matrix:")
    for i in range(m):
        row = []
        for j in range(n+1):
            row.append(value(model.u[i, j]))
        print(row)

    print("Objective function value:", value(model.max_distance))

    distance = 0
    for i in range(m):
      distance += np.sum(courier_matrices[i] * d)

    print("The sum of distances is:", distance)

else:
    print("No solution found.")

# # Check for errors during solver initialization
# if result.solver.status != SolverStatus.ok:
#     # raise an exception
#     raise ValueError("Solver failed to initialize: " + result.solver.message)

# # Check for non-optimal termination conditions
# if result.solver.termination_condition not in [TerminationCondition.optimal, TerminationCondition.feasible]:
#     print("Solver terminated with non-optimal status:", result.solver.termination_condition)

# # Print the solution
# else:
#     items = [[] for i in range(m)]
#     for i in range(m):
#         for j in range(n):
#             if model.x[i,j].value == 1:
#                 items[i].append(j)
#     print(items)
#     print("Objective function value: ", value(model.obj))


# print(result)
# print(model.display())

Items: [[0], [1], [3], [6], [4], [2, 5, 9], [8], [7]]
Assignment Matrix:
[[1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0]]
Courier 0:
[0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0]
Courier 1:
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0]
Courier 2:
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 